## Część 2

In [147]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="secret.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()
query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,ES_CT,2020-08-01,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
1,ES_CT,2020-05-15,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
2,ES_CT,2022-05-11,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
3,ES_CT,2022-07-14,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
4,ES_CT,2021-10-31,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
5,ES_CT,2022-07-17,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
6,ES_CT,2020-05-14,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
7,ES_CT,2021-03-06,ChIJ8_UwhdxbpBIRUMijIeD6AAE,Q5705,nuts/ES51,ES,Spain,ES,ESP,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.537778 41.8375)
8,US_AL,2020-09-05,ChIJdf5LHzR_hogR6czIUzU0VV4,Q173,geoId/01,US,United States of America,US,USA,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-86.7 32.7)
9,US_AL,2021-03-06,ChIJdf5LHzR_hogR6czIUzU0VV4,Q173,geoId/01,US,United States of America,US,USA,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-86.7 32.7)


## Część 3

3.1. Sprawdź, ile jest zapisanych wierszy z danymi. 

In [148]:
query = ("""
select count(*) as liczba_wierszy from bigquery-public-data.covid19_open_data.covid19_open_data;
""")

query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_wierszy
0,22756333


 3.2. Sprawdź, ile krajów jest uwzględnionych w danych. 



In [149]:
query = ("""
select count(*) as liczba_krajow FROM (SELECT DISTINCT country_name FROM bigquery-public-data.covid19_open_data.covid19_open_data);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_krajow
0,246


 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.



Dzienne informacje dla krajów raportowane są z podziałem na części opisywane przez aggregation_level.
Dane odnośnie osób hospitalizowanych, zaszczepionych, zmarłych i testowanych raportowane są oddzielnie dla poszczególnych przedziałów wiekowych oraz z podziałem na płeć. 
Dane zawierają również podstawowe informacje o poszczególnych krajach, takie jak populacja, powierzchnia, wilgotność czy przewidywana długość życia.

 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.  



In [150]:
print(df1['population'].dtype)
print(df1['nurses_per_1000'].dtype)
print(df1['new_tested'].dtype)

Int64
float64
object


 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.



In [151]:
query = ("""
select min(date) as poczatek_przedzialu, max(date) as koniec_przedzialu FROM bigquery-public-data.covid19_open_data.covid19_open_data;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df
# co to znaczy nowych?

,poczatek_przedzialu,koniec_przedzialu
0,2020-01-01,2022-09-17


In [152]:
query = ("""
select min(date) as poczatek_przedzialu_zachorowan, max(date) as koniec_przedzialu_zachorowan FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed is not null and new_confirmed <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zachorowan,koniec_przedzialu_zachorowan
0,2020-01-01,2022-09-15


In [153]:
query = ("""
select min(date) as poczatek_przedzialu_smierci, max(date) as koniec_przedzialu_smierci FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_deceased is not null and new_deceased <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_smierci,koniec_przedzialu_smierci
0,2020-01-02,2022-09-15


In [154]:
query = ("""
select min(date) as poczatek_przedzialu_zaszczepionych, max(date) as koniec_przedzialu_zaszczepionych FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_persons_vaccinated is not null and new_persons_vaccinated <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zaszczepionych,koniec_przedzialu_zaszczepionych
0,2020-12-07,2022-09-14


 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 



In [155]:
# Sprawdzenie czy wystepuja wartosci ujemne
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed < 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,102199


In [156]:
#Sprawdzenie czy wystepuja duplikaty
query = ("""
select COUNT(*), location_key, date FROM bigquery-public-data.covid19_open_data.covid19_open_data group by location_key, date having count(*) > 1;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_,location_key,date


In [157]:
# Sprawdzenie czy kluczowe kolumny zawieraja nulle
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where date is null or country_name is null or location_key is null or iso_3166_1_alpha_3 is null;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,991


In [158]:
# Sprawdzenie z jakich krajow posiadamy najwiecej danych
query = ("""
select country_name, count(*) as count from bigquery-public-data.covid19_open_data.covid19_open_data group by country_name order by count desc limit 10;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,country_name,count
0,Brazil,5565456
1,United States of America,3255435
2,Mexico,2475518
3,Peru,1883891
4,Israel,1472626
5,Spain,1385418
6,Colombia,1145596
7,India,747214
8,Indonesia,544059
9,Argentina,540095


In [159]:
# sprawdzenie czy w kolumnach cumulative wystepuja wartosci mniejsze niz w kolumnach new
query = ("""
select distinct location_key, country_name from bigquery-public-data.covid19_open_data.covid19_open_data where (new_confirmed > cumulative_confirmed or new_deceased > cumulative_deceased or cast(new_tested as int) > cumulative_tested or new_persons_vaccinated > cumulative_persons_vaccinated or new_persons_fully_vaccinated > cumulative_persons_fully_vaccinated or new_vaccine_doses_administered > cumulative_vaccine_doses_administered);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,location_key,country_name
0,CH_TG,Switzerland
1,CH,Switzerland
2,PL_24,Poland
3,LI,Liechtenstein
4,MY,Malaysia
5,PL_14,Poland
6,PL_22,Poland
7,PL_26,Poland
8,PL_16,Poland
9,PL_30,Poland


## Część 4

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.



In [160]:
query = ("""
SELECT distinct 
    country_code,
    country_name, 
    iso_3166_1_alpha_2, #imo useless
    iso_3166_1_alpha_3, 
    population, 
    population_male, 
    population_female, 
    population_density, 
    human_development_index, 
    gdp_usd, 
    gdp_per_capita_usd, 
    latitude, 
    longitude, 
    area_sq_km, 
    health_expenditure_usd, 
    out_of_pocket_health_expenditure_usd 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    country_code IS NOT NULL AND 
    country_name IS NOT NULL AND 
    iso_3166_1_alpha_2 IS NOT NULL AND 
    iso_3166_1_alpha_3 IS NOT NULL AND 
    population IS NOT NULL AND 
    population_male IS NOT NULL AND 
    population_female IS NOT NULL AND 
    population_density IS NOT NULL AND 
    human_development_index IS NOT NULL AND 
    gdp_usd IS NOT NULL AND 
    gdp_per_capita_usd IS NOT NULL AND 
    latitude IS NOT NULL AND 
    longitude IS NOT NULL AND 
    area_sq_km IS NOT NULL AND 
    health_expenditure_usd IS NOT NULL AND 
    out_of_pocket_health_expenditure_usd IS NOT NULL
    AND aggregation_level = 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1.to_csv("countries_data.csv", index=False)
df1

,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,population_density,human_development_index,gdp_usd,gdp_per_capita_usd,latitude,longitude,area_sq_km,health_expenditure_usd,out_of_pocket_health_expenditure_usd
0,RO,Romania,RO,ROU,19414458,9500450,9914008,83.100,0.811,241475766400,12390,46.000000,25.000000,238400,555.104736,113.730110
1,BB,Barbados,BB,BRB,287371,139084,148287,668.305,0.800,5209000000,18148,13.170000,-59.552500,430,1183.836182,548.158508
2,CD,Democratic Republic of the Congo,CD,COD,89561404,44710172,44851232,39.506,0.457,47319624204,545,-2.880000,23.656000,2344860,19.431646,7.796894
3,CO,Colombia,CO,COL,50882884,24984564,25898320,45.861,0.747,323802808108,6432,4.000000,-73.250000,1141748,459.197571,74.887901
4,GM,Gambia,GM,GMB,2416664,1198540,1218124,238.801,0.460,1763819047,751,13.500000,-15.500000,11300,23.272326,5.127221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,PL,Poland,PL,POL,37972812,18380376,19592436,123.600,0.843,585705873800,15222,52.000000,19.000000,312680,906.820129,209.020157
171,TD,Chad,TD,TCD,16425859,8200170,8225689,13.045,0.404,11314951342,709,15.466667,19.400000,1284000,29.730864,17.250788
172,TZ,Tanzania,TZ,TZA,59734213,29851108,29883105,67.435,0.538,63177068174,1122,-6.307000,34.854000,947300,33.916576,8.172585
173,VU,Vanuatu,VU,VUT,307150,155690,151460,25.197,0.603,917058850,3058,-16.633331,168.016669,12190,105.666527,9.468102


 4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.



In [161]:
query = ("""select distinct date, country_name, new_confirmed, cumulative_confirmed, cast(new_tested as int) as new_tested,  cumulative_tested from bigquery-public-data.covid19_open_data.covid19_open_data WHERE
  date IS NOT NULL
  AND country_name IS NOT NULL
  AND new_confirmed IS NOT NULL
  AND cumulative_confirmed IS NOT NULL
  AND new_tested IS NOT NULL
  AND cumulative_tested IS NOT NULL
  AND new_confirmed <= cumulative_confirmed
  and cast(new_tested as int) <= cumulative_tested
  AND new_confirmed >= 0
  AND cast(new_tested as int) >= 0
  AND cumulative_confirmed >= 0
  AND cumulative_tested >= 0
  AND aggregation_level = 0
  """)
query_job = client.query(query)
query_result = query_job.result()
df2 = query_result.to_dataframe()
df2.to_csv("covid_cases_stats.csv", index=False)
df2

,date,country_name,new_confirmed,cumulative_confirmed,new_tested,cumulative_tested
0,2022-03-11,Belarus,1931,939514,19447,12620704
1,2022-03-04,Djibouti,0,15550,912,286712
2,2021-05-15,Suriname,121,11828,306,17522
3,2021-04-26,Jordan,1815,706355,18853,6649640
4,2021-06-01,Azerbaijan,156,334288,9649,3523638
...,...,...,...,...,...,...
73510,2021-06-02,Zimbabwe,61,39092,1817,557596
73511,2020-08-28,Zimbabwe,18,6406,1114,98386
73512,2020-08-31,Zimbabwe,62,6559,1013,100837
73513,2021-01-21,Zimbabwe,476,30523,3829,292140


 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.



In [162]:
query = ("""
SELECT 
    date,  
    country_name,  
    new_deceased, 
    cumulative_deceased 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    new_deceased IS NOT NULL AND 
    cumulative_deceased IS NOT NULL AND
    new_deceased <= cumulative_deceased AND
    new_deceased >= 0
    AND cumulative_deceased >= 0
    AND aggregation_level = 0

    ;

""")
query_job = client.query(query)
query_result = query_job.result()
df3 = query_result.to_dataframe()
df3.to_csv("covid_mortality.csv", index=False)
df3

,date,country_name,new_deceased,cumulative_deceased
0,2021-11-03,Argentina,12,116693
1,2020-09-16,Argentina,351,19025
2,2022-01-06,Argentina,71,117946
3,2022-05-05,Argentina,5,128800
4,2020-10-23,Argentina,353,32443
...,...,...,...,...
227569,2020-12-06,Zimbabwe,3,294
227570,2020-02-04,Zimbabwe,0,5
227571,2021-06-17,Zimbabwe,9,1656
227572,2022-08-27,Zimbabwe,0,5593


 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.



In [163]:
import numpy as np

query = ("""
SELECT 
    date,  
    country_name, 
    new_persons_vaccinated, 
    cumulative_persons_vaccinated, 
    new_persons_fully_vaccinated, 
    cumulative_persons_fully_vaccinated, 
    new_vaccine_doses_administered, 
    cumulative_vaccine_doses_administered 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL 
    AND aggregation_level = 0

    ;

""")
query_job = client.query(query)
query_result = query_job.result()
df4 = query_result.to_dataframe()
df4.replace(np.nan, 0, inplace=True)
df4.to_csv("covid_vaccination.csv", index=False)
df4

,date,country_name,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered
0,2020-12-16,Andorra,0,0,0,0,0,0
1,2020-08-16,Andorra,0,0,0,0,0,0
2,2021-07-24,Andorra,0,0,0,0,0,0
3,2022-01-06,Andorra,0,0,0,0,0,0
4,2020-06-04,Andorra,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
243781,2022-04-25,Zimbabwe,7441,5835677,4744,3641341,17537,10071251
243782,2022-01-09,Zimbabwe,2986,4168084,4698,3186507,9166,7373616
243783,2022-03-16,Zimbabwe,5210,4410552,4134,3438471,13745,8027283
243784,2021-06-15,Zimbabwe,1162,694728,4113,418728,5275,1113456


 4.5. Zdefiniuj własny dodatkowy przypadek. Chcemy dowiedzieć się ile jest pielęgniarek i lekarzy w poszczególnych krajach.



In [164]:
query = ("""
SELECT distinct
    country_name, 
    nurses_per_1000,
    physicians_per_1000
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    country_name IS NOT NULL AND 
    nurses_per_1000 >= 0 AND
    physicians_per_1000 >= 0 AND
    aggregation_level = 0
    ;
""")
query_job = client.query(query)
query_result = query_job.result()
df5 = query_result.to_dataframe()
df5.to_csv("covid_vaccination.csv", index=False)
df5

,country_name,nurses_per_1000,physicians_per_1000
0,Burkina Faso,0.8829,0.0847
1,Chile,13.3248,2.5912
2,China,2.6621,1.9798
3,Finland,14.7374,3.8118
4,Gabon,2.9460,0.6819
...,...,...,...
159,Nauru,7.6636,1.3462
160,Peru,2.4398,1.3048
161,Sudan,0.6952,0.2618
162,Suriname,2.7569,1.2101


## Część 5
Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [166]:
# df1 + gdp + world_countries + df5
# df2 + df3 + df4
merged_df = df2.merge(df3, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df4, on=["country_name", "date"], how="inner")
merged_df = merged_df.dropna()
merged_df.to_csv("covid_combined_data.csv", index=False)
merged_df

,date,country_name,new_confirmed,cumulative_confirmed,new_tested,cumulative_tested,new_deceased,cumulative_deceased,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered
0,2022-03-11,Belarus,1931,939514,19447,12620704,14,6647,0,0,0,0,0,0
1,2022-03-04,Djibouti,0,15550,912,286712,0,189,0,0,0,0,0,0
2,2021-05-15,Suriname,121,11828,306,17522,1,224,1057,49961,614,1612,1671,51573
3,2021-04-26,Jordan,1815,706355,18853,6649640,47,8707,0,0,0,0,0,0
4,2021-06-01,Azerbaijan,156,334288,9649,3523638,5,4926,51940,1404898,7559,907817,59499,2312715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73329,2021-06-02,Zimbabwe,61,39092,1817,557596,5,1604,4239,682240,4694,357952,8933,1040192
73330,2020-08-28,Zimbabwe,18,6406,1114,98386,1,198,0,0,0,0,0,0
73331,2020-08-31,Zimbabwe,62,6559,1013,100837,1,205,0,0,0,0,0,0
73332,2021-01-21,Zimbabwe,476,30523,3829,292140,45,962,0,0,0,0,0,0


## Część 6
Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach

In [167]:
world_countries = pd.read_csv("world_countries.csv")
world_countries = world_countries.rename(columns={"CCA3": "iso_3166_1_alpha_3", "Country/Territories": "country_name", "Area (km²)": "area_sq_km", "Density (per km²)": "population_density"})

gdp = pd.read_csv("GDP.csv")
gdp = gdp.rename(columns={"Country Code": "iso_3166_1_alpha_3", "Country Name": "country_name", "Value": "gdp_usd", "Year":"year"})
df1["year"] = pd.to_datetime(merged_df["date"]).dt.year
merged_df2 = df1.merge(world_countries, on="iso_3166_1_alpha_3", how="inner")
merged_df2 = merged_df2.merge(df5, on=['country_name'])
pivot_gdp = gdp.pivot(index='country_name', columns='year', values='gdp_usd')
pivot_gdp.columns = ['gdp_' + str(column) for column in pivot_gdp.columns]
merged_df2 = merged_df2.merge(pivot_gdp, on="country_name", how="inner")
merged_df2.to_csv("countries_combined_data.csv", index=False)
merged_df2


,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,population_density_x,human_development_index,gdp_usd,...,gdp_2007,gdp_2008,gdp_2009,gdp_2010,gdp_2011,gdp_2012,gdp_2013,gdp_2014,gdp_2015,gdp_2016
0,RO,Romania,RO,ROU,19414458,9500450,9914008,83.100,0.811,241475766400,...,1.715367e+11,2.081816e+11,1.674229e+11,1.679981e+11,1.853629e+11,1.716646e+11,1.915490e+11,1.994935e+11,1.779111e+11,1.875920e+11
1,BB,Barbados,BB,BRB,287371,139084,148287,668.305,0.800,5209000000,...,4.523750e+09,4.607300e+09,4.434050e+09,4.461650e+09,4.660900e+09,4.656350e+09,4.612500e+09,4.608350e+09,4.584150e+09,4.529050e+09
2,CO,Colombia,CO,COL,50882884,24984564,25898320,45.861,0.747,323802808108,...,2.074165e+11,2.439824e+11,2.338217e+11,2.870182e+11,3.354152e+11,3.696597e+11,3.801919e+11,3.781957e+11,2.915196e+11,2.824626e+11
3,GN,Guinea,GN,GIN,13132792,6353234,6779558,53.446,0.459,13590281808,...,6.281696e+09,6.963962e+09,6.716818e+09,6.861546e+09,6.785189e+09,7.387104e+09,8.376668e+09,8.791354e+09,8.767144e+09,8.200248e+09
4,GT,Guatemala,GT,GTM,17915567,8827109,9088458,167.185,0.627,76710385879,...,3.411311e+10,3.913689e+10,3.773399e+10,4.133860e+10,4.765484e+10,5.038845e+10,5.385106e+10,5.872232e+10,6.376760e+10,6.876326e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,PL,Poland,PL,POL,37972812,18380376,19592436,123.600,0.843,585705873800,...,4.292496e+11,5.338158e+11,4.403466e+11,4.792579e+11,5.287251e+11,5.002840e+11,5.242012e+11,5.450759e+11,4.772796e+11,4.713644e+11
133,TD,Chad,TD,TCD,16425859,8200170,8225689,13.045,0.404,11314951342,...,8.638711e+09,1.035193e+10,9.253484e+09,1.065771e+10,1.215638e+10,1.236807e+10,1.294985e+10,1.392222e+10,1.088880e+10,9.600761e+09
134,TZ,Tanzania,TZ,TZA,59734213,29851108,29883105,67.435,0.538,63177068174,...,2.150174e+10,2.736839e+10,2.857378e+10,3.140791e+10,3.387863e+10,3.908775e+10,4.433346e+10,4.819722e+10,4.562832e+10,4.734007e+10
135,VU,Vanuatu,VU,VUT,307150,155690,151460,25.197,0.603,917058850,...,5.264283e+08,6.079586e+08,6.100666e+08,7.008043e+08,7.921497e+08,7.817029e+08,8.017876e+08,8.149543e+08,7.424321e+08,7.735029e+08
